In [27]:
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files
import string
import xml.etree.ElementTree as et

Georgia
-------
Election results from the Georgia Secretary of State Elections Division (https://sos.ga.gov/index.php/Elections/current_and_past_elections_results). Presidential recount results from the Georgia Secretary of State Elections Division via Reuters.

Precinct shapefile primarily from the Georgia General Assembly Reapportionment Office (http://www.legis.ga.gov/Joint/reapportionment/en-US/default.aspx). The following counties instead use shapefiles from the U.S. Census Bureau's 2020 Redistricting Data Program Phase 2 release: Cobb, DeKalb, Gwinnett. Forsyth and Fulton use shapefiles sourced from the respective counties.

Three of the four VTDs in Chattahoochee County are comprised of Fort Benning. However, the county only reports one polling location for all voters, including residents of Fort Benning that vote within the county. The four Chattahoochee County VTDs have therefore been merged in the shapefile.

The following additional modifications reflect changes made prior to the 2020 general election:

Barrow: Merge 2/15, 3/12, 4/14, 5/7, 6/10/13, 8/9, 11/16; Adjust new 2/13 boundary
Bartow: Split Cassville/Hamilton Crossing
Candler: Merge Candler/Metter as Jack Strickland Comm Center
Chatham: Split 7-7/8-16, 7-12/7-16; Realign 7-06C/7-07C
Chatooga: Split Cloudland/Teloga along ridgeline that marks boundary between them with the USGS Topographic Contour shapefile
Clayton: Split Ellenswood 1/2, Jonesboro 1/17/19, Lovejoy 3/6/7, Morrow 3/11, 5/10, Oak 3/5 
Cobb: Split Bells Ferry 3/4, Dobbins 1/2, Marietta 3A/3B, Smyrna 3A/3B
Columbia: Split Bessie Thomas/2nd Mt Moriah, Harlem Branch/Harlem Senior Ctr; Merge Blanchard Park/MTZ Col FD;  Align multiple precincts with county maps
Coweta: Merge Arts Centre/Jefferson Parkway as Newnan Centre
Fulton: Merge CP07A/CP07D, CH01/CH04B, SS29A/SS29B, UC031/UC035
DeKalb: Split Clarkston/Clarkston Comm Ctr; Realign Decatur/Oakhurst; Align precincts with Atlanta, Brookhaven, Decatur, Tucker city limits 
Gwinnett: Adjust Baycreek F/G, Berkshire J/M, Cates D/F, Garners C/B, Lawrenceville G/N, Pinckneyville S/T, Rockbridge A/G
Lowndes: Split Northgate Assembly/Trinity, Jaycee/Mt Calvary/Northside/VSU
Oconee: Merge Annex/City Hall; Align City Hall with Watkinsville city limits
Paulding: Reorganize 12 precincts into 19 precincts as redrawn in 2019
Randolph: Merge Carnegie/Cuthbert-Courthouse, 4th District/Fountain Bridge/Shellman
Troup: Split Mountville between Gardner Newman/Hogansville/Rosemont; Align multiple precincts with county maps
Towns: Merge Macedonia/Tate City

Note that the leading zeros in the Paulding County precinct IDs are included in some election reports and omitted in others. The shapefile includes the leading zeros consistent with the voter file.

G20PRERTRU - Donald J. Trump (Republican Party)
G20PREDBID - Joseph R. Biden (Democratic Party)
G20PRELJOR - Jo Jorgensen (Libertarian Party)

C20PRERTRU - Donald J. Trump (Republican Party)
C20PREDBID - Joseph R. Biden (Democratic Party)
C20PRELJOR - Jo Jorgensen (Libertarian Party)

G20USSRPER - David A. Perdue (Republican Party)
G20USSDOSS - Jon Ossoff (Democratic Party)
G20USSLHAZ - Shane Hazel (Libertarian Party)

S20USSRLOE - Kelly Loeffler (Republican Party)
S20USSRCOL - Doug Collins (Republican Party)
S20USSRGRA - Derrick E. Grayson (Republican Party)
S20USSRJAC - Annette Davis Jackson (Republican Party)
S20USSRTAY - Kandiss Taylor (Republican Party)
S20USSRJOH - A. Wayne Johnson (Republican Party)
S20USSDWAR - Raphael Warnock (Democratic Party)
S20USSDJAC - Deborah Jackson (Democratic Party)
S20USSDLIE - Matt Lieberman (Democratic Party)
S20USSDJOH - Tamara Johnson-Shealey (Democratic Party)
S20USSDJAM - Jamesia James (Democratic Party)
S20USSDSLA - Joy Felicia Slade (Democratic Party)
S20USSDWIN - Richard Dien Winfield (Democratic Party)
S20USSDTAR - Ed Tarver (Democratic Party)
S20USSLSLO - Brian Slowinski (Libertarian Party)
S20USSGFOR - John Fortuin (Green Party)
S20USSIBUC - Allen Buckley (Independent)
S20USSIBAR - Al Bartell (Independent)
S20USSISTO - Valencia Stovall (Independent)
S20USSIGRE - Michael Todd Greene (Independent)

G20PSCRSHA - Jason Shaw (Republican Party)
G20PSCDBRY - Robert G. Bryant (Democratic Party)
G20PSCLMEL - Elizabeth Melton (Libertarian Party)

G20PSCRMCD - Lauren Bubba McDonald, Jr. (Republican Party)
G20PSCDBLA - Daniel Blackman (Democratic Party)
G20PSCLWIL - Nathan Wilson (Libertarian Party)


In [28]:
loaded_counties = os.listdir("./raw-from-source/Non_Recount_Results/")
z=[]
for locale in loaded_counties:
    if locale.endswith('.xml'):
        file_string = "./raw-from-source/Non_Recount_Results/"+locale
        xtree = et.parse(file_string)
        xroot = xtree.getroot()
        store_list = []
        county_area = xroot.findall(".//Region")
        #county = county_area.attrib.get('text')
        for i in county_area:
            county = i.text
        contests = xroot.findall(".//Contest")
        for i in contests:
            contest = i.attrib.get('text')
            lower = i.findall("./Choice")
            for j in lower:
                choice = j.attrib.get('text')
                lower_2 = j.findall("./VoteType")
                for k in lower_2:
                    voting_method = k.attrib.get('name')
                    lower_3 = k.findall("./Precinct")
                    for l in lower_3:
                        precinct_name = l.attrib.get('name')
                        num_votes = l.attrib.get('votes')
                        z.append([county,contest,choice,voting_method,precinct_name,num_votes])
                        #print(x)
                    #print("")
                #print("")
            #print("")
dfcols = ['county','contest','choice','voting_method','precinct','num_votes']
df_general = pd.DataFrame(z,columns=dfcols)
print(df_general)

       county                                            contest  \
0       Crisp                     President of the United States   
1       Crisp                     President of the United States   
2       Crisp                     President of the United States   
3       Crisp                     President of the United States   
4       Crisp                     President of the United States   
...       ...                                                ...   
581519  Evans  Claxton Sunday Package Sales Referendum - Special   
581520  Evans  Claxton Sunday Package Sales Referendum - Special   
581521  Evans  Claxton Sunday Package Sales Referendum - Special   
581522  Evans  Claxton Sunday Package Sales Referendum - Special   
581523  Evans  Claxton Sunday Package Sales Referendum - Special   

                           choice           voting_method  \
0       Donald J. Trump (I) (Rep)      Election Day Votes   
1       Donald J. Trump (I) (Rep)      Election Day Votes   


In [29]:
loaded_counties = os.listdir("./raw-from-source/Recount_Results/")
z=[]
for locale in loaded_counties:
    if locale.endswith('.xml'):
        file_string = "./raw-from-source/Recount_Results/"+locale
        xtree = et.parse(file_string)
        xroot = xtree.getroot()
        store_list = []
        county_area = xroot.findall(".//Region")
        #county = county_area.attrib.get('text')
        for i in county_area:
            county = i.text
        contests = xroot.findall(".//Contest")
        for i in contests:
            contest = i.attrib.get('text')
            lower = i.findall("./Choice")
            for j in lower:
                choice = j.attrib.get('text')
                lower_2 = j.findall("./VoteType")
                for k in lower_2:
                    voting_method = k.attrib.get('name')
                    lower_3 = k.findall("./Precinct")
                    for l in lower_3:
                        precinct_name = l.attrib.get('name')
                        num_votes = l.attrib.get('votes')
                        z.append([county,contest,choice,voting_method,precinct_name,num_votes])
dfcols = ['county','contest','choice','voting_method','precinct','num_votes']
df_recount = pd.DataFrame(z,columns=dfcols)

In [30]:
print(df_recount)
print(df_recount["contest"].unique())

       county                         contest                     choice  \
0      Oconee  President of the United States  Donald J. Trump (I) (Rep)   
1      Oconee  President of the United States  Donald J. Trump (I) (Rep)   
2      Oconee  President of the United States  Donald J. Trump (I) (Rep)   
3      Oconee  President of the United States  Donald J. Trump (I) (Rep)   
4      Oconee  President of the United States  Donald J. Trump (I) (Rep)   
...       ...                             ...                        ...   
31867  Thomas  President of the United States         Jo Jorgensen (Lib)   
31868  Thomas  President of the United States         Jo Jorgensen (Lib)   
31869  Thomas  President of the United States         Jo Jorgensen (Lib)   
31870  Thomas  President of the United States         Jo Jorgensen (Lib)   
31871  Thomas  President of the United States         Jo Jorgensen (Lib)   

            voting_method         precinct num_votes  
0      Election Day Votes       

In [31]:
contest_changes_dict = {'President of the United States':'President-Recount',
 'President of the United States/Presidentede los Estados Unidos':'President-Recount'}

df_recount["contest"] = df_recount["contest"].map(contest_changes_dict).fillna(df_recount["contest"])

In [119]:
print(df_general.shape)
print(df_recount.shape)

ga_election = pd.concat([df_general,df_recount])
print(len(ga_election["county"].unique()))

(581524, 6)
(31872, 6)
159


In [120]:
office_list = ['President of the United States','US Senate (Perdue)','US Senate (Loeffler) - Special',
              'Public Service Commission District 1','Public Service Commission District 4',
              'US Senate (Loeffler) - Special Election','President-Recount','President of the United States/Presidentede los Estados Unidos',
              'Public Service Commission Dist 1/Comisionado de Servicio Público Dist 1',
       'Public Service Commission Dist 4/Comisionado de Servicio Público Dist 4','US Senate (Perdue)/Senado de los EE.UU. (Perdue)',
       'US Senate (Loeffler) - Special/Senado de los EE.UU. (Loeffler) - Especial',]

ga_election = ga_election[ga_election["contest"].isin(office_list)]

In [121]:
print(ga_election["contest"].unique())

['President of the United States' 'US Senate (Perdue)'
 'US Senate (Loeffler) - Special' 'Public Service Commission District 1'
 'Public Service Commission District 4'
 'US Senate (Loeffler) - Special Election'
 'President of the United States/Presidentede los Estados Unidos'
 'US Senate (Perdue)/Senado de los EE.UU. (Perdue)'
 'US Senate (Loeffler) - Special/Senado de los EE.UU. (Loeffler) - Especial'
 'Public Service Commission Dist 1/Comisionado de Servicio Público Dist 1'
 'Public Service Commission Dist 4/Comisionado de Servicio Público Dist 4'
 'President-Recount']


In [122]:
contest_changes_dict = {'US Senate (Loeffler) - Special Election':'US Senate (Loeffler) - Special',
                        'US Senate (Loeffler) - Special/Senado de los EE.UU. (Loeffler) - Especial':'US Senate (Loeffler) - Special',
                        'US Senate (Perdue)/Senado de los EE.UU. (Perdue)':'US Senate (Perdue)',
                        'President of the United States/Presidentede los Estados Unidos':'President of the United States',
                        'Public Service Commission Dist 1/Comisionado de Servicio Público Dist 1':'Public Service Commission District 1',
                        'Public Service Commission Dist 4/Comisionado de Servicio Público Dist 4':'Public Service Commission District 4'}

ga_election["contest"] = ga_election["contest"].map(contest_changes_dict).fillna(ga_election["contest"])
print(ga_election["contest"].unique())

['President of the United States' 'US Senate (Perdue)'
 'US Senate (Loeffler) - Special' 'Public Service Commission District 1'
 'Public Service Commission District 4' 'President-Recount']


In [123]:
ga_election["num_votes"]=ga_election["num_votes"].astype(int)
sum(ga_election["num_votes"])

29581664

In [126]:
print(ga_election["choice"].unique())

['Donald J. Trump (I) (Rep)' 'Joseph R. Biden (Dem)' 'Jo Jorgensen (Lib)'
 'David A. Perdue (I) (Rep)' 'Jon Ossoff (Dem)' 'Shane Hazel (Lib)'
 'Al Bartell (Ind)' 'Allen Buckley (Ind)' 'Doug Collins (Rep)'
 'John Fortuin (Grn)' 'Derrick E. Grayson (Rep)'
 'Michael Todd Greene (Ind)' 'Annette Davis Jackson (Rep)'
 'Deborah Jackson (Dem)' 'Jamesia James (Dem)' 'A. Wayne Johnson (Rep)'
 'Tamara Johnson-Shealey (Dem)' 'Matt Lieberman (Dem)'
 'Kelly Loeffler (I) (Rep)' 'Joy Felicia Slade (Dem)'
 'Brian Slowinski (Lib)' 'Valencia Stovall (Ind)' 'Ed Tarver (Dem)'
 'Kandiss Taylor (Rep)' 'Raphael Warnock (Dem)'
 'Richard Dien Winfield (Dem)' 'Jason Shaw (I) (Rep)'
 'Robert G. Bryant (Dem)' 'Elizabeth Melton (Lib)'
 'Lauren Bubba McDonald, Jr. (I) (Rep)' 'Daniel Blackman (Dem)'
 'Nathan Wilson (Lib)' 'Matt Lierberman (Dem)']


In [127]:
cand_change_dict = {'Matt Lierberman (Dem)':'Matt Lieberman (Dem)'}
ga_election["choice"] = ga_election["choice"].map(cand_change_dict).fillna(ga_election["choice"])

In [128]:
fips_file = pd.read_csv("./raw-from-source/FIPS/US_FIPS_Codes.csv")
fips_file = fips_file[fips_file["State"]=="Georgia"]
fips_file["FIPS County"] = fips_file["FIPS County"].astype(str)
fips_file["FIPS County"] = fips_file["FIPS County"].str.zfill(3)
fips_file["County Name"] = fips_file["County Name"].replace("De Kalb","DeKalb")
fips_dict = dict(zip(fips_file['County Name'], fips_file['FIPS County']))

In [129]:
ga_election['county_fips'] = ga_election['county'].map(fips_dict).fillna(ga_election['county'])
ga_election['county_fips'] = ga_election['county_fips'].astype(str)
ga_election['county_fips'] = ga_election['county_fips'].str.zfill(3)
ga_election["unique_ID"]=ga_election["county_fips"]+ga_election["precinct"]
ga_election["choice"]=ga_election["choice"]+ga_election["contest"]
ga_election=pd.pivot_table(ga_election,index=["unique_ID","county","county_fips","precinct"],columns=["choice"],values=['num_votes'],aggfunc=sum)
ga_election = ga_election.fillna(0)
ga_election.columns = ga_election.columns.droplevel(0)
ga_election = ga_election.reset_index()

In [130]:
column_changes_dict = {'A. Wayne Johnson (Rep)US Senate (Loeffler) - Special':'S20USSRJOH',
       'Al Bartell (Ind)US Senate (Loeffler) - Special':'S20USSIBAR',
       'Allen Buckley (Ind)US Senate (Loeffler) - Special':'S20USSIBUC',
       'Annette Davis Jackson (Rep)US Senate (Loeffler) - Special':'S20USSRJAC',
       'Brian Slowinski (Lib)US Senate (Loeffler) - Special':'S20USSLSLO',
       'Daniel Blackman (Dem)Public Service Commission District 4':'G20PSCDBLA',
       'David A. Perdue (I) (Rep)US Senate (Perdue)':'G20USSRPER',
       'Deborah Jackson (Dem)US Senate (Loeffler) - Special':'S20USSDJAC',
       'Derrick E. Grayson (Rep)US Senate (Loeffler) - Special':'S20USSRGRA',
       'Donald J. Trump (I) (Rep)President of the United States':'G20PRERTRU',
       'Donald J. Trump (I) (Rep)President-Recount':'C20PRERTRU',
       'Doug Collins (Rep)US Senate (Loeffler) - Special':'S20USSRCOL',
       'Ed Tarver (Dem)US Senate (Loeffler) - Special':'S20USSDTAR',
       'Elizabeth Melton (Lib)Public Service Commission District 1':'G20PSCLMEL',
       'Jamesia James (Dem)US Senate (Loeffler) - Special':'S20USSDJAM',
       'Jason Shaw (I) (Rep)Public Service Commission District 1':'G20PSCRSHA',
       'Jo Jorgensen (Lib)President of the United States':'G20PRELJOR',
       'Jo Jorgensen (Lib)President-Recount':'C20PRELJOR',
       'John Fortuin (Grn)US Senate (Loeffler) - Special':'S20USSGFOR',
       'Jon Ossoff (Dem)US Senate (Perdue)':'G20USSDOSS',
       'Joseph R. Biden (Dem)President of the United States':'G20PREDBID',
       'Joseph R. Biden (Dem)President-Recount':'C20PREDBID',
       'Joy Felicia Slade (Dem)US Senate (Loeffler) - Special':'S20USSDSLA',
       'Kandiss Taylor (Rep)US Senate (Loeffler) - Special':'S20USSRTAY',
       'Kelly Loeffler (I) (Rep)US Senate (Loeffler) - Special':'S20USSRLOE',
       'Lauren Bubba McDonald, Jr. (I) (Rep)Public Service Commission District 4':'G20PSCRMCD',
       'Matt Lieberman (Dem)US Senate (Loeffler) - Special':'S20USSDLIE',
       'Michael Todd Greene (Ind)US Senate (Loeffler) - Special':'S20USSIGRE',
       'Nathan Wilson (Lib)Public Service Commission District 4':'G20PSCLWIL',
       'Raphael Warnock (Dem)US Senate (Loeffler) - Special':'S20USSDWAR',
       'Richard Dien Winfield (Dem)US Senate (Loeffler) - Special':'S20USSDWIN',
       'Robert G. Bryant (Dem)Public Service Commission District 1':'G20PSCDBRY',
       'Shane Hazel (Lib)US Senate (Perdue)':'G20USSLHAZ',
       'Tamara Johnson-Shealey (Dem)US Senate (Loeffler) - Special':'S20USSDJOH',
       'Valencia Stovall (Ind)US Senate (Loeffler) - Special':'S20USSISTO'}

In [131]:
ga_election = ga_election.rename(columns=column_changes_dict)

In [132]:
print(ga_election.columns)

Index(['unique_ID', 'county', 'county_fips', 'precinct', 'S20USSRJOH',
       'S20USSIBAR', 'S20USSIBUC', 'S20USSRJAC', 'S20USSLSLO', 'G20PSCDBLA',
       'G20USSRPER', 'S20USSDJAC', 'S20USSRGRA', 'G20PRERTRU', 'C20PRERTRU',
       'S20USSRCOL', 'S20USSDTAR', 'G20PSCLMEL', 'S20USSDJAM', 'G20PSCRSHA',
       'G20PRELJOR', 'C20PRELJOR', 'S20USSGFOR', 'G20USSDOSS', 'G20PREDBID',
       'C20PREDBID', 'S20USSDSLA', 'S20USSRTAY', 'S20USSRLOE', 'G20PSCRMCD',
       'S20USSDLIE', 'S20USSIGRE', 'G20PSCLWIL', 'S20USSDWAR', 'S20USSDWIN',
       'G20PSCDBRY', 'G20USSLHAZ', 'S20USSDJOH', 'S20USSISTO'],
      dtype='object', name='choice')


In [75]:
vest_ga_20 = gp.read_file("./raw-from-source/VEST/ga_2020/ga_2020.shp")

In [89]:
print(vest_ga_20.shape)

(2679, 44)


       DISTRICT CTYSOSID PRECINCT_I PRECINCT_N CTYNAME CTYNUMBER CTYNUMBER2  \
1329   051XFTPU     None       None       None    None      None       None   
1447  029FTSTEW     None       None       None    None      None       None   
2125  215FTBEN2     None       None       None    None      None       None   
2128  215FTBEN4     None       None       None    None      None       None   
2146  215FTBEN3     None       None       None    None      None       None   
2229  215ZZZZZZ     None       None       None    None      None       None   
2230  215FTBEN1     None       None       None    None      None       None   

     FIPS2  G20PRERTRU  G20PREDBID  ...  S20USSIBAR  S20USSISTO  S20USSIGRE  \
1329  None           0           0  ...           0           0           0   
1447  None           0           0  ...           0           0           0   
2125  None           0           0  ...           0           0           0   
2128  None           0           0  ...           0

None


In [151]:
ga_election['FIPS2'] = ga_election["county_fips"]

In [147]:
print(len()

159


In [155]:
data_columns = ['G20PRERTRU', 'G20PREDBID',
       'G20PRELJOR', 'C20PRERTRU', 'C20PREDBID', 'C20PRELJOR', 'G20USSRPER',
       'G20USSDOSS', 'G20USSLHAZ', 'S20USSRLOE', 'S20USSRCOL', 'S20USSRGRA',
       'S20USSRJAC', 'S20USSRTAY', 'S20USSRJOH', 'S20USSDWAR', 'S20USSDJAC',
       'S20USSDLIE', 'S20USSDJOH', 'S20USSDJAM', 'S20USSDSLA', 'S20USSDWIN',
       'S20USSDTAR', 'S20USSLSLO', 'S20USSGFOR', 'S20USSIBUC', 'S20USSIBAR',
       'S20USSISTO', 'S20USSIGRE', 'G20PSCRSHA', 'G20PSCDBRY', 'G20PSCLMEL',
       'G20PSCRMCD', 'G20PSCDBLA', 'G20PSCLWIL']


for race in data_columns:
    if (sum(vest_ga_20[race])-sum(ga_election[race]) != 0):
        print(race+" has a difference of "+str(sum(vest_ga_20[race])-sum(ga_election[race]))+" votes")
        print("\tVEST: "+str(sum(vest_ga_20[race]))+" votes")
        print("\tSOURCES: "+str(sum(frame[race]))+" votes")
    else:
        print(race+":EQUAL")

G20PRERTRU:EQUAL
G20PREDBID:EQUAL
G20PRELJOR:EQUAL
C20PRERTRU:EQUAL
C20PREDBID:EQUAL
C20PRELJOR:EQUAL
G20USSRPER:EQUAL
G20USSDOSS:EQUAL
G20USSLHAZ:EQUAL
S20USSRLOE:EQUAL
S20USSRCOL:EQUAL
S20USSRGRA:EQUAL
S20USSRJAC:EQUAL
S20USSRTAY:EQUAL
S20USSRJOH:EQUAL
S20USSDWAR:EQUAL
S20USSDJAC:EQUAL
S20USSDLIE:EQUAL
S20USSDJOH:EQUAL
S20USSDJAM:EQUAL
S20USSDSLA:EQUAL
S20USSDWIN:EQUAL
S20USSDTAR:EQUAL
S20USSLSLO:EQUAL
S20USSGFOR:EQUAL
S20USSIBUC:EQUAL
S20USSIBAR:EQUAL
S20USSISTO:EQUAL
S20USSIGRE:EQUAL
G20PSCRSHA:EQUAL
G20PSCDBRY:EQUAL
G20PSCLMEL:EQUAL
G20PSCRMCD:EQUAL
G20PSCDBLA:EQUAL
G20PSCLWIL:EQUAL


In [157]:
print("Counties with differences printed below:")
diff_counties=[]
for i in data_columns:
    diff = vest_ga_20.groupby(["FIPS2"]).sum()[i]-ga_election.groupby(["FIPS2"]).sum()[i]
    print(diff)
    for val in diff[diff != 0].index.values.tolist():
        if val not in diff_counties:
            diff_counties.append(val)
    if len(diff[diff != 0]!=0):
        print(i)
        print(diff[diff != 0].to_string(header=False))
print("")
print("All other races in all counties are equal")

Counties with differences printed below:
FIPS2
001    0
003    0
005    0
007    0
009    0
      ..
313    0
315    0
317    0
319    0
321    0
Name: G20PRERTRU, Length: 159, dtype: int64
FIPS2
001    0
003    0
005    0
007    0
009    0
      ..
313    0
315    0
317    0
319    0
321    0
Name: G20PREDBID, Length: 159, dtype: int64
FIPS2
001    0
003    0
005    0
007    0
009    0
      ..
313    0
315    0
317    0
319    0
321    0
Name: G20PRELJOR, Length: 159, dtype: int64
FIPS2
001    0
003    0
005    0
007    0
009    0
      ..
313    0
315    0
317    0
319    0
321    0
Name: C20PRERTRU, Length: 159, dtype: int64
FIPS2
001    0
003    0
005    0
007    0
009    0
      ..
313    0
315    0
317    0
319    0
321    0
Name: C20PREDBID, Length: 159, dtype: int64
FIPS2
001    0
003    0
005    0
007    0
009    0
      ..
313    0
315    0
317    0
319    0
321    0
Name: C20PRELJOR, Length: 159, dtype: int64
FIPS2
001    0
003    0
005    0
007    0
009    0
      ..
313  

In [158]:
for i in list(vest_ga_20['FIPS2'].unique()):
    if i not in (list(ga_election['FIPS2'].unique())):
        print(i)

print(vest_ga_20[vest_ga_20['FIPS2'].isna()])

None
       DISTRICT CTYSOSID PRECINCT_I PRECINCT_N CTYNAME CTYNUMBER CTYNUMBER2  \
1329   051XFTPU     None       None       None    None      None       None   
1447  029FTSTEW     None       None       None    None      None       None   
2125  215FTBEN2     None       None       None    None      None       None   
2128  215FTBEN4     None       None       None    None      None       None   
2146  215FTBEN3     None       None       None    None      None       None   
2229  215ZZZZZZ     None       None       None    None      None       None   
2230  215FTBEN1     None       None       None    None      None       None   

     FIPS2  G20PRERTRU  G20PREDBID  ...  S20USSIBAR  S20USSISTO  S20USSIGRE  \
1329  None           0           0  ...           0           0           0   
1447  None           0           0  ...           0           0           0   
2125  None           0           0  ...           0           0           0   
2128  None           0           0  ...       

In [159]:
print(vest_ga_20)

      DISTRICT  CTYSOSID PRECINCT_I          PRECINCT_N   CTYNAME CTYNUMBER  \
0       215122    215122        122       FIRST AFRICAN  MUSCOGEE       106   
1       215108    215108        108  ST MARK/HEIFERHORN  MUSCOGEE       106   
2       057031    057031        031           R T JONES  CHEROKEE        28   
3       057033    057033        033             SALACOA  CHEROKEE        28   
4        01506     01506         06              CENTER    BARTOW         8   
...        ...       ...        ...                 ...       ...       ...   
2674  121SC14A  121SC14A      SC14A               SC14A    FULTON        60   
2675    12109B    12109B        09B                 09B    FULTON        60   
2676   121CH05   121CH05       CH05                CH05    FULTON        60   
2677  121FA01D  121FA01D      FA01D               FA01D    FULTON        60   
2678     07707     07707         07        FISCHER ROAD    COWETA        38   

     CTYNUMBER2 FIPS2  G20PRERTRU  G20PREDBID  ... 

In [172]:
ga_election["precinct"] = ga_election["precinct"].str.strip().str.upper()
ga_election["unique_ID"]=ga_election["county_fips"]+"-"+ga_election["precinct"]
vest_ga_20["unique_ID"]=vest_ga_20["FIPS2"]+"-"+vest_ga_20["PRECINCT_I"]

join_attempt_1 = pd.merge(ga_election,vest_ga_20,how="outer",on="unique_ID",indicator=True)
print(join_attempt_1["_merge"].value_counts())

right_only    2067
left_only     2044
both           612
Name: _merge, dtype: int64


In [173]:
ga_election["precinct"] = ga_election["precinct"].str.strip().str.upper()
ga_election["unique_ID"]=ga_election["county_fips"]+"-"+ga_election["precinct"]
vest_ga_20["unique_ID"]=vest_ga_20["FIPS2"]+"-"+vest_ga_20["PRECINCT_N"]

join_attempt_1 = pd.merge(ga_election,vest_ga_20,how="outer",on="unique_ID",indicator=True)
print(join_attempt_1["_merge"].value_counts())

both          2545
right_only     134
left_only      111
Name: _merge, dtype: int64


In [174]:
left_only = join_attempt_1[join_attempt_1["_merge"]=="left_only"]
right_only = join_attempt_1[join_attempt_1["_merge"]=="right_only"]

left_only.to_csv("./source_only.csv")
right_only.to_csv("./VEST_only.csv")

In [176]:
example_csv = pd.read_csv("./source_vest_id_changes.csv")
example_fips_dict = dict(zip(example_csv["source_ID"],example_csv["vest_ID"]))

In [177]:
ga_election["unique_ID"] = ga_election["unique_ID"].map(example_fips_dict).fillna(ga_election["unique_ID"])

In [178]:
join_attempt_2 = pd.merge(ga_election,vest_ga_20,how="outer",on="unique_ID",indicator=True)
print(join_attempt_2["_merge"].value_counts())

left_only = join_attempt_1[join_attempt_1["_merge"]=="left_only"]
right_only = join_attempt_1[join_attempt_1["_merge"]=="right_only"]

both          2658
right_only      23
left_only        0
Name: _merge, dtype: int64


In [194]:
empty_precincts = []
for index, row in right_only.iterrows():
    for race in data_columns:
        keep=True
        if (row[race+"_y"]!=0):
            keep = False
    if(keep):
        empty_precincts.append(row["DISTRICT"])
print(len(empty_precincts))  

23


## Shapefiles 